# Test Script for MongoDB 

In [1]:
#Import Dependancies
import pandas as pd
import pymongo
from pymongo import MongoClient

In [2]:
# Connection string to MongoDB
conn='mongodb://localhost:27017'

# Making a Connection with MongoClient
client = MongoClient(conn)

# create database
db = client.deathDB

# create first collection
col_one = db.Heart_Failure_Records

# create second collection
col_two = db.Stroke_Records

In [3]:
# Read Heart Data into a DataFrame
heart_df = pd.read_csv('heart.csv')
heart_df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
#Group by death event and find median age
death_event = heart_df.groupby('DEATH_EVENT')['age'].mean()
death_event

DEATH_EVENT
0    58.761906
1    65.215281
Name: age, dtype: float64

In [5]:
heart_dict = heart_df.to_dict('records')
col_one.insert_many(heart_dict)

In [6]:
#Filter heart failure records by gender
Heart_Failure_Records = db.Heart_Failure_Records.find()   
# Create empty list and fill with collection records
data=[]
for item in Heart_Failure_Records:
    heart_dict={}
    heart_dict['age']=item['age']
    heart_dict['anaemia']=item['anaemia']
    heart_dict['diabetes']=item['diabetes']
    heart_dict['ejection_fraction']=item['ejection_fraction']
    heart_dict['high_blood_pressure']=item['high_blood_pressure']
    heart_dict['platelets']=item['platelets']
    heart_dict['serum_creatinine']=item['serum_creatinine']
    heart_dict['serum_sodium']=item['serum_sodium']
    heart_dict['sex']=item['sex']
    heart_dict['smoking']=item['smoking']
    heart_dict['time']=item['time']
    heart_dict['Death Event']=item['DEATH_EVENT']
    data.append(heart_dict)
# Filter by Male
data_df = pd.DataFrame(data)
sex_filter = data_df.loc[:,'sex']== 1
sex_df = data_df.loc[sex_filter,:]
sex_df.head()

,age,anaemia,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,Death Event
0,75.0,0,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,0,20,0,210000.00,1.9,137,1,0,7,1
5,90.0,1,0,40,1,204000.00,2.1,132,1,1,8,1


In [7]:
# Read Stroke Data into a DataFrame
stroke_df = pd.read_csv('stroke.csv')
stroke_df = stroke_df.set_index('id')
stroke_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [8]:
# Convert Stroke Data to Dictionary and Push to MongoDB Collection
stroke_dict = stroke_df.to_dict('records')
col_two.insert_many(stroke_dict)

In [9]:
# Filter stroke records by gender
stroke_records = db.Stroke_Records.find()  

# Create empty list and fill with collection records
data=[]
for item in stroke_records:
    stroke_dict={}
    stroke_dict['gender']=item['gender']
    stroke_dict['age']=item['age']
    stroke_dict['hypertension']=item['hypertension']
    stroke_dict['heart_disease']=item['heart_disease']
    stroke_dict['ever_married']=item['ever_married']
    stroke_dict['work_type']=item['work_type']
    stroke_dict['Residence_type']=item['Residence_type']
    stroke_dict['avg_glucose_level']=item['avg_glucose_level']
    stroke_dict['bmi']=item['bmi']
    stroke_dict['smoking_status']=item['smoking_status']
    stroke_dict['stroke']=item['stroke']
    data.append(stroke_dict)
    
# Filter by Male
data_df = pd.DataFrame(data)
gender_filter = data_df.loc[:,'gender']=='Male'
gender_df = data_df.loc[gender_filter,:]
gender_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
13,Male,78.0,0,1,Yes,Private,Urban,219.84,NaN,Unknown,1


In [10]:
# Read a couple lines from heart collection in mongodb
# Display items in MongoDB collection
results = db.Heart_Failure_Records.find()

for result in results[:5]:
    print(result)

{'_id': ObjectId('604cff83fe48c2fa1f925540'), 'age': 75.0, 'anaemia': 0, 'creatinine_phosphokinase': 582, 'diabetes': 0, 'ejection_fraction': 20, 'high_blood_pressure': 1, 'platelets': 265000.0, 'serum_creatinine': 1.9, 'serum_sodium': 130, 'sex': 1, 'smoking': 0, 'time': 4, 'DEATH_EVENT': 1}
{'_id': ObjectId('604cff83fe48c2fa1f925541'), 'age': 55.0, 'anaemia': 0, 'creatinine_phosphokinase': 7861, 'diabetes': 0, 'ejection_fraction': 38, 'high_blood_pressure': 0, 'platelets': 263358.03, 'serum_creatinine': 1.1, 'serum_sodium': 136, 'sex': 1, 'smoking': 0, 'time': 6, 'DEATH_EVENT': 1}
{'_id': ObjectId('604cff83fe48c2fa1f925542'), 'age': 65.0, 'anaemia': 0, 'creatinine_phosphokinase': 146, 'diabetes': 0, 'ejection_fraction': 20, 'high_blood_pressure': 0, 'platelets': 162000.0, 'serum_creatinine': 1.3, 'serum_sodium': 129, 'sex': 1, 'smoking': 1, 'time': 7, 'DEATH_EVENT': 1}
{'_id': ObjectId('604cff83fe48c2fa1f925543'), 'age': 50.0, 'anaemia': 1, 'creatinine_phosphokinase': 111, 'diabetes'

In [11]:
# Read a couple lines from stroke collection in mongodb
# Display items in MongoDB collection
results = db.Stroke_Records.find()

for result in results[:5]:
    print(result)

{'_id': ObjectId('604cff84fe48c2fa1f92566b'), 'gender': 'Male', 'age': 67.0, 'hypertension': 0, 'heart_disease': 1, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 228.69, 'bmi': 36.6, 'smoking_status': 'formerly smoked', 'stroke': 1}
{'_id': ObjectId('604cff84fe48c2fa1f92566c'), 'gender': 'Female', 'age': 61.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type': 'Self-employed', 'Residence_type': 'Rural', 'avg_glucose_level': 202.21, 'bmi': nan, 'smoking_status': 'never smoked', 'stroke': 1}
{'_id': ObjectId('604cff84fe48c2fa1f92566d'), 'gender': 'Male', 'age': 80.0, 'hypertension': 0, 'heart_disease': 1, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Rural', 'avg_glucose_level': 105.92, 'bmi': 32.5, 'smoking_status': 'never smoked', 'stroke': 1}
{'_id': ObjectId('604cff84fe48c2fa1f92566e'), 'gender': 'Female', 'age': 49.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type':

In [12]:
client.close()